In [1]:
import glob 
import pandas as pd 
import xml.etree.ElementTree as ET 
import os
from datetime import datetime

logfile    = "output/dealership_logfile.txt" 
targetfile = "output/dealership_transformed_data.csv"  

In [2]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe

def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        car_model = person.find("car_model").text 
        year_of_manufacture = int(person.find("year_of_manufacture").text)       
        price = float(person.find("price").text) 
        fuel = person.find("fuel").text 
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True) 
        return dataframe
    
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("output/dealership_logfile.txt","a") as f: 
        f.write(timestamp + ',' + message + '\n')

In [3]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) 
    #for csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
    #for json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    #for xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
    return extracted_data

def transform(data):
    data['price'] = round(data.price, 2)
    return data

def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

In [4]:
log("ETL Job Started")
log("Extract phase Started")
extracted_data = extract() 
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")

In [5]:
glob.glob("*.csv")

['used_car_prices1.csv', 'used_car_prices2.csv', 'used_car_prices3.csv']

In [6]:
# def extract(data_source):
#     #do...
#     return extracted_data

# def transform(data):
#     #do...
#     return data

# def load(target, data_to_load):
#     data_to_load --> targetfile